<a href="https://colab.research.google.com/github/shahabbadihi/IT/blob/master/sentimentalComments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sklearn
import pandas as pd
import requests
import io
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from spacy.lang.fa import stop_words
from string import punctuation, printable, digits
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.base import BaseEstimator, TransformerMixin

file_url = "https://raw.githubusercontent.com/shahabbadihi/IT/master/2-p9vcb5bb.xlsx"
response = requests.get(file_url)

dest = 'local-file.xlsx'

with open(dest, "wb") as filee:
  filee.write(response.content)

df = pd.read_excel(dest)

df.head(100)

df.nunique()

df.groupby('user_id').count().sort_values(by=['product_id'])

tdf = df
tdf['cart_id'] = 0
tdf.head(100)


user_list = np.array(tdf['user_id']).tolist()
user_list = list(set(user_list)) 
print(user_list)

comments = tdf[['product_id','user_id','comment']]
comments.head(100)

def load_stopwords():
    file_url = "https://raw.githubusercontent.com/shahabbadihi/IT/master/fa_stop_words.txt"
    response = requests.get(file_url)
    with open("fa-stop-words.txt", "wb") as filee:
        filee.write(response.content)

    f = open("fa-stop-words.txt", "r", encoding='utf8')
    stopwords = f.read()
    stopwords = stopwords.split('\n')
    stopwords = set(stopwords)
    custom_stop_words = {'آنكه','آيا','بدين','براين','بنابر','میشه','میکنه','باشه','سلام','میکشه','اونی',''}
    stopwords = stopwords | stop_words.STOP_WORDS | custom_stop_words
    # excluding space
    stopwords = list(stopwords)[1:]
    unwanted_num = {'خوش','بهتر','بد','خوب','نیستم','عالی','نیست','فوق','بهترین'} 
    stopwords = [ele for ele in stopwords if ele not in unwanted_num] 
    return stopwords
print (load_stopwords())

class Preprocess(BaseEstimator, TransformerMixin):
    def __init__(self, stop_words):
        self.stop_words = stop_words
    def fit(self, X, y=None):
        return self
    def transform(self, corpus):
        res = []
        for data in corpus:
            if not self.stop_words:
                self.stop_words = set([])
            ## ensure working with string
            doc = str(data)
            # First remove punctuation form string
            PUNCT_DICT = {ord(punc): None for punc in punctuation+'،'}
            doc = doc.translate(PUNCT_DICT)
            # remove numbers
            doc = doc.translate({ord(k): None for k in digits})
            tokens = doc.split()
            tokens = [t for t in tokens if len(t) > 1]
            res.append(' '.join(w for w in tokens if w not in self.stop_words))
        return res



Preprocess(load_stopwords()).transform([comments.comment[10]])

text_clf = Pipeline([
    ('prep', Preprocess(load_stopwords())),
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB()),
])



[1179650, 3932163, 4587529, 4718603, 6422542, 3276814, 3538958, 917524, 5636118, 7077910, 8126490, 4325404, 3014685, 1310755, 1179684, 1179690, 917547, 786475, 1179693, 7864370, 8257587, 3014709, 655415, 6029368, 7864376, 4587576, 8257593, 3145788, 4325437, 7995447, 4194367, 7995461, 7733318, 7077961, 2883657, 7733326, 2359378, 1441874, 2752595, 2883669, 7995476, 2752596, 3932250, 2883675, 7864412, 7995486, 7340132, 4587621, 3407974, 2752615, 1441896, 2490473, 786542, 655470, 6291571, 6946933, 6684790, 5243002, 4063356, 655485, 7864445, 5243004, 4587649, 6291588, 2883723, 5243020, 5243019, 3539086, 655506, 786579, 7864470, 655511, 3014809, 8126622, 1179807, 2490528, 524449, 7078051, 1310884, 3276965, 3539109, 6291623, 8126632, 3670186, 7995564, 6815919, 7864497, 8519857, 7340212, 7995575, 6684864, 7864517, 3014857, 655562, 917708, 5505229, 3932366, 6553806, 8126673, 3014867, 7864535, 7995610, 6947039, 7864545, 8126691, 1179880, 1310954, 1442028, 6684909, 8519922, 2621684, 4980982, 4063

In [3]:
for idx, row in tdf.iterrows():
  if row['recommend'] == 'recommended':	
     tdf.loc[idx, 'rating'] = 10
  if row['recommend'] == 'not_recommended':
    tdf.loc[idx, 'rating'] = 1
  if row['recommend'] == 'no_idea':
    tdf.loc[idx, 'rating'] = 5
tdf.head(100)


,product_id,product_title,title_en,user_id,likes,dislikes,verification_status,recommend,title,comment,advantages,disadvantages,cart_id,rating
0,3692,ماوس بی‌سیم لاجیتک مدل M325,IT,989472,0,0,verified,\N,NaN,واقعا عالیه. من که ازش خیلی راضیم,NaN,NaN,0,NaN
1,90213,شارژر همراه شیاومی مدل NDY-02-AN با ظرفیت 1000...,AC,3862150,4,1,verified,recommended,واقعاً عالیه,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,"[""عمر طولانی\r"",""افت بسیار کم میزان شارژ\r"",""ا...","[""ندارد""]",0,10.0
2,59473,یدک پولیشر میکروفایبر مهسان مدل 20119,HW,626843,1,0,verified,not_recommended,خیلی سخت حوله اش در میاد,گیره های فلزی خیلی سخت تا میشوند و لذا حوله را...,NaN,NaN,0,1.0
3,120499,گوشی موبایل هوآوی آنر مدل 5X KIW-L21 دو سیم‌کارت,MO,786887,6,11,verified,no_idea,گوشی مقرون به صرفه,همه چیز در رابطه با ظاهر این گوشی بسیار خوب اس...,"[""صفحه نمایش پرنور و با کیفیت\r"",""کیفیت بالای ...","[""کیفیت پایین اسپیکر\r"",""حاشیه خالی زیر صفحه ن...",0,5.0
4,67200,شارژر همراه شیائومی مدل Mi ظرفیت 5000 میلی آمپ...,AC,854531,19,4,verified,no_idea,ابعاد، استحکام و نگهداری شارژ مناسب,اگر ظرفیتش براتون کافیه حتما بخرید._x000D_\nیه...,"[""ظریف و زیبا و باریک\r"",""بدنه محکم و با دوام""]","[""ظرفیت متوسط""]",0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,163713,میکروسکوپ سولومارک مدل HM900,AV,1244756,1,1,verified,no_idea,برای بچه های.کنجکاو.تا دبیرستان.خوبه,برای کنجکاوی و.سرگرمی خوبه دوستی که گفته بود ک...,NaN,NaN,0,5.0
96,147614,شارژر فندکی ریمکس مدل RCC303,AC,3400245,1,2,verified,\N,بخرید,هم سرعت شارژ و هم محکم ماندن در جای فندک عالیه,NaN,NaN,0,NaN
97,56736,دریل چکشی بوش مدل PSB 530 RE,TC,770375,9,0,verified,recommended,بعد از 2 سال,من 2 سال پیش نظر دادم سال 95 و الان 97 باز میگ...,"[""كارامد\r"",""لدون لرزش و شل شدگي\r"",""بسيار پرق...","[""نداشتن اچار روی دسته""]",0,10.0
98,189665,کالسکه ارابه مدل Stripes,TS,2756636,6,2,verified,recommended,قیمت خوبی داره,متاسفانه چرخ های جلو به خوبی به اطراف نمی چرخد...,NaN,"[""چرخ های جلو به خوبی به اطراف نمی چرخد""]",0,10.0


In [8]:


comments = tdf[['product_id','user_id','comment','rating']]

comments = comments.dropna(subset=['rating'])

train_X, test_X, train_y, test_y = train_test_split(comments['comment'].values, comments['rating'].values, stratify=comments['rating'])
train_X.shape, train_y.shape, test_X.shape, test_y.shape
text_clf.fit(train_X, train_y)
print(text_clf.predict(train_X))

[10. 10. 10. ...  1. 10. 10.]
